In [ ]:
import pandas as pd

In [ ]:
song_df = pd.read_excel('song_df.xlsx', index_col=0)
lyrics_df = pd.read_excel('song_lyrics_df.xlsx', index_col=0)
song_album_df = pd.read_excel('song_album_df.xlsx', index_col=0)
song_artist_df = pd.read_excel('song_artist_df.xlsx', index_col=0)
song_genre_df = pd.read_excel('song_genre_df.xlsx', index_col=0)

### 19금 노래 제거

In [ ]:
for st in song_df['title']:
    if st[:3] == '19금':
        song_df.drop(song_df[song_df['title'] == st].index, inplace=True)

song_df = song_df.drop_duplicates('song_id')
song_df.reset_index(drop=True)
song_df.to_excel('./song_df.xlsx')


def drop_none_song(song_df, df):
    for si in df['song_id']:
        if song_df[song_df['song_id'] == si].empty:
            df.drop(df[df['song_id'] == si].index, inplace=True)

    return df.reset_index(drop=True)

In [ ]:
song_album_df = drop_none_song(song_df, song_album_df)
song_artist_df = drop_none_song(song_df, song_artist_df)
song_genre_df = drop_none_song(song_df, song_genre_df)
lyrics_df = drop_none_song(song_df, lyrics_df)

In [ ]:
song_album_df.to_excel('./song_album_df.xlsx')
song_artist_df.to_excel('./song_artist_df.xlsx')
song_genre_df.to_excel('./song_genre_df.xlsx')
lyrics_df.to_excel('./song_lyrics_df.xlsx')

### 자연어 처리용 데이터셋 생성

In [ ]:
for i in range(len(lyrics_df)):
    song_id = lyrics_df.loc[i, 'song_id']
    lyrics_df.loc[i, 'title'] = song_df[song_df['song_id'] == song_id].title.item()

lyrics_df.to_excel('lyrics_data.xlsx')

In [ ]:
from konlpy.tag import Okt

lyrics_df = pd.read_excel('lyrics_data.xlsx',index_col=0)

lyrics = lyrics_df.lyrics[8000].replace(' \\n', '')

In [ ]:
okt = Okt()

lyrics_word = okt.morphs(lyrics, norm=True)

stopword = ['도', '는', '다', '의', '가', '이', '은', ',', '될',
            '한', '에', '하', '고', '을', '를', '인', '.', '로',
            '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

word_token = [w for w in lyrics_word if not w in stopword]

word_token

## 감성대화

### 모듈 import

In [ ]:
import json
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import notebook
import pandas as pd
import numpy as np

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

### 원천 데이터 불러오기

In [ ]:
with open('./018.감성대화/Training_221115_add/라벨링데이터/감성대화말뭉치(최종데이터)_Training/감성대화말뭉치(최종데이터)_Training.json', encoding='utf-8') as json_f:
    json_sen = json.load(json_f)

with open('./018.감성대화/Validation_221115_add/라벨링데이터/감성대화말뭉치(최종데이터)_Validation/감성대화말뭉치(최종데이터)_Validation.json', encoding='utf-8') as json_t_f:
    json_t_sen = json.load(json_t_f)

### 새 데이터 프레임 생성

In [ ]:
emo_df = pd.DataFrame(columns=['sentense','emotion'])
emo_test_df = pd.DataFrame(columns=['sentense', 'emotion'])

### DF에 데이터 인풋 및 CSV 저장

In [ ]:
for sen in json_sen:
    emo = sen['profile']['emotion']['type']
    conv = sen['talk']['content']
    cnt = 0
    for t in conv:
        if t[0] == 'H' and conv[t]:
            emo_df.loc[len(emo_df),:] = [conv[t], emo]

In [ ]:
emo_df.to_csv('./emo_df.csv', index=True)

In [ ]:
for sen in notebook.tqdm(json_t_sen):
    emo = sen['profile']['emotion']['type']
    conv = sen['talk']['content']
    for t in conv:
        if t[0] == 'H' and conv[t]:
            emo_test_df.loc[len(emo_test_df),:] = [conv[t], emo]

In [ ]:
emo_test_df.to_csv('./emo_test_df.csv', index=True)

### 감성 정리

In [3]:
import pandas as pd
from tqdm.notebook import tqdm

emo_df = pd.read_csv('emo_df.csv', index_col=0)
emo_test_df = pd.read_csv('emo_test_df.csv', index_col=0)

분노 = (10, 11, 12, 13, 14, 16)
악의 = (15, 17, 18, 19, 57, 58)
슬픔 = (20, 21, 23, 24, 53, 51)
절망 = (22, 25, 26, 27, 28, 29)
당황 = (32, 34, 35, 38, 50, 59)
불안 = (30, 31, 33, 36, 37, 39)
열등 = (41, 43, 45, 46, 47, 54, 52)
상처 = (40, 42, 44, 48, 49, 55, 56)
사랑 = (60, 61, 62, 64, 65)
편안 = (63, 66, 67, 68, 69)

In [4]:
def emotion_orz(df, name):
    for i in tqdm(range(len(df))):
        e = int(df['emotion'][i][1:])
        if e in 분노:
            e = '0'
        elif e in 악의:
            e = '1'
        elif e in 슬픔:
            e = '2'
        elif e in 절망:
            e = '3'
        elif e in 당황:
            e = '4'
        elif e in 불안:
            e = '5'
        elif e in 열등:
            e = '6'
        elif e in 상처:
            e = '7'
        elif e in 사랑:
            e = '8'
        elif e in 편안:
            e = '9'
        
        df.loc[i, 'emotion'] = e

    df.to_csv(f'{name}_newemo.csv')

emotion_orz(emo_df, 'emo_df')
emotion_orz(emo_test_df, 'emo_test_df')

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

### 토크나이저?

In [5]:
train_data = []
test_data = []

for i in notebook.tqdm(range(len(emo_df))):
    train_data.append([emo_df.loc[i, 'sentense'], emo_df.loc[i, 'emotion']])

for i in notebook.tqdm(range(len(emo_test_df))):
    test_data.append([emo_test_df.loc[i, 'sentense'], emo_df.loc[i, 'emotion']])

  0%|          | 0/145959 [00:00<?, ?it/s]

KeyboardInterrupt: 

# 여긴 나중에

### 불용어 제거 및 트레인 셋으로 워드백 생성

In [ ]:
t_train = []
t_test = []

stopword = ['도', '는', '다', '의', '가', '이', '은', ',', '될',
            '한', '에', '하', '고', '을', '를', '인', '.', '로',
            '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

for sentense in notebook.tqdm(emo_df['sentense']):
    word_back = okt.morphs(sentense, norm=True)
    word_back = [word for word in word_back if not word in stopword]
    t_train.append(word_back)

for sentense in notebook.tqdm(emo_test_df['sentense']):
    word_back = okt.morphs(sentense, norm=True)
    word_back = [word for word in word_back if not word in stopword]
    t_test.append(word_back)

In [ ]:
json_object = []
for i in notebook.tqdm(range(len(t_train))):
    tmp = {}
    tmp = {'sentense' : t_train[i], 'emotion' : emo_df.iloc[i, 1]}
    json_object.append(tmp)

with open('./sentense_train.json', 'w', encoding='utf-8') as f:
    json.dump(json_object, f, indent=4, ensure_ascii=False)

In [ ]:
json_test_object = []
for i in notebook.tqdm(range(len(t_test))):
    tmp = {}
    tmp = {'sentense' : t_test[i], 'emotion' : emo_test_df.iloc[i, 1]}
    json_test_object.append(tmp)

with open('./sentense_test.json', 'w', encoding='utf-8') as f:
    json.dump(json_test_object, f, indent=4, ensure_ascii=False)

### 토크나이저

In [ ]:
with open('./sentense_train.json', 'r', encoding='utf-8') as train_f:
    t_train_json = json.load(train_f)
    
with open('./sentense_test.json', 'r', encoding='utf-8') as test_f:
    t_test_json = json.load(test_f)


t_train = []
for sen in t_train_json:
    t_train.append(sen['sentense'])
    
t_test = []
for sen in t_test_json:
    t_test.append(sen['sentense'])

In [ ]:
tokenizer = Tokenizer()

In [ ]:
tokenizer.fit_on_texts(t_train)

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt) * 100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq) * 100)

In [ ]:
t_train1 = tokenizer.texts_to_sequences(t_train)

t_test1 = tokenizer.texts_to_sequences(t_test)

In [4]:
cnt = 0
for i in tqdm(range(10000000)):
    cnt += 1

  0%|          | 0/10000000 [00:00<?, ?it/s]

In [14]:
import pandas as pd
from tqdm.notebook import tqdm

In [28]:
train_df = pd.read_csv('emo_df.csv', index_col=0)
test_df = pd.read_csv('emo_test_df.csv', index_col=0)

In [29]:
for i in tqdm(range(len(train_df))):
    train_df.loc[i, 'emotion'] = train_df.loc[i, 'emotion'][1:2]

for i in tqdm(range(len(test_df))):
    test_df.loc[i, 'emotion'] = test_df.loc[i, 'emotion'][1:2]

  0%|          | 0/145959 [00:00<?, ?it/s]

  0%|          | 0/17965 [00:00<?, ?it/s]

In [30]:
train_df.to_csv('emo_df_6emo.csv')
test_df.to_csv('emo_df_test_6emo.csv')